In [1]:
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = r'.\dataset\dota_dataset'
OUTPUT_PATH = r'.\dataset\dota_split_dataset'

In [55]:
def split_image(filename):
    print(filename)
    image_file = os.path.join(DATA_PATH, 'images', filename + '.png')
    label_file = os.path.join(DATA_PATH, 'labels', filename + '.txt')
    image = cv2.imread(image_file)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label_list = []
    with tf.gfile.GFile(label_file, 'r') as f:
        labels = f.readlines()
        for label in labels:
            if label.find('plane') != -1 or label.find('helicopter') != -1:
                label = label.strip().split(r' ')
                label_list.append({
                    'startX':min(int(float(label[0])),int(float(label[4]))),
                    'startY':min(int(float(label[1])),int(float(label[5]))),
                    'endX':max(int(float(label[0])),int(float(label[4]))),
                    'endY':max(int(float(label[1])),int(float(label[5]))),
                    'type':label[8]
                })
    xList = [ x for x in range(0, image.shape[1] - 500, 250)]
    yList = [ y for y in range(0, image.shape[0] - 500, 250)]
    xList.append(image.shape[1] - 500)
    yList.append(image.shape[0] - 500)
    count = 0
    canvas = image.copy()
    for x in xList:
        for y in yList:
            exist_flag = False
            image_output_path = os.path.join(OUTPUT_PATH, 'images', filename + '-' + str(count) + '.jpg')
            label_output_path = os.path.join(OUTPUT_PATH, 'labels', filename + '-' + str(count) + '.txt')
            visualization_output_path = os.path.join(OUTPUT_PATH, 'visualization', filename + '-' + str(count) + '.jpg')
            for label in label_list: 
                area = (label['endX'] - label['startX']) * (label['endY'] - label['startY'])
                middleX = (label['endX'] + label['startX']) / 2
                middleY = (label['endY'] + label['startY']) / 2
                startX = max(x, label['startX'])
                startY = max(y, label['startY'])
                endX = min(x + 500, label['endX'])
                endY = min(y + 500, label['endY'])
                public_area = (endX - startX) * (endY - startY)
                if x < middleX < x + 500 and y < middleY < y + 500 and public_area / area > 0.75:
                    exist_flag = True
                    print((count, x, y, x+500, y+500))
                    print(startX-x, startY-y, endX-x, endY-y)
                    with open(label_output_path, 'a') as f:
                        pass
                        f.write('{type} {startX} {startY} {endX} {endY}\n'.format(type=label['type'], startX=startX-x, startY=startY-y, endX=endX-x, endY=endY-y))
                                
                    cv2.rectangle(canvas, (startX, startY),(endX, endY),(0, 255, 255), 3)
            count += 1
            if exist_flag:
                cv2.rectangle(canvas, (x, y), (x+500, y+500), (255, 255, 0), 3)
                cv2.imwrite(image_output_path, image[y:y+500, x:x+500, 0:3])
                # 通过读取输出文件生成可视化图便于检验
                test_image = cv2.imread(image_output_path)
                with open(label_output_path, 'r') as f:
                    test_labels = f.readlines()
                    for test_label in test_labels:
                        test_label = test_label.split(' ')
                        cv2.rectangle(test_image, 
                                      (int(test_label[1]), int(test_label[2])), 
                                      (int(test_label[3]), int(test_label[4])), 
                                      (0, 255, 255), 3)
                        if test_label[0] == 'plane': 
                            label_length = 45 
                        else:
                            label_length = 60
                        cv2.rectangle(test_image, 
                                      (int(test_label[1])-3, int(test_label[2])-20), 
                                      (int(test_label[1])+label_length, int(test_label[2])), 
                                      (0, 255, 255), -1)
                        cv2.putText(test_image, test_label[0], (int(test_label[1]), int(test_label[2])-7), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)       
                cv2.imwrite(visualization_output_path, test_image)
    cv2.imwrite(os.path.join(OUTPUT_PATH, 'visualization', filename + '.jpg'), canvas)
    print('包含样本%s，标记样本%s'%(len(label_list), count))                
                    

In [56]:
for i in os.walk(os.path.join(DATA_PATH, 'labels')):
    label_files = i[2]
    for label_file in label_files:
        label_file_path = os.path.join(DATA_PATH, 'labels', label_file)
        with tf.gfile.GFile(label_file_path, 'r') as lf:
            all_in_one = lf.readlines()
            for line in all_in_one:
                if line.find('plane') != -1 or line.find('helicopter') != -1:
                    split_image(label_file.split('.')[0])
                    break

P0000
(59, 500, 3750, 1000, 4250)
298 165 476 332
(60, 500, 4000, 1000, 4500)
183 182 364 345
(60, 500, 4000, 1000, 4500)
131 322 313 485
(61, 500, 4250, 1000, 4750)
131 72 313 235
(80, 750, 3500, 1250, 4000)
175 242 356 429
(81, 750, 3750, 1250, 4250)
48 165 226 332
(81, 750, 3750, 1250, 4250)
175 0 356 179
(105, 1000, 4250, 1500, 4750)
354 245 500 399
(106, 1000, 4500, 1500, 5000)
354 0 500 149
(125, 1250, 3750, 1750, 4250)
259 237 390 381
(126, 1250, 4000, 1750, 4500)
259 0 390 131
(127, 1250, 4250, 1750, 4750)
104 245 258 399
(128, 1250, 4500, 1750, 5000)
104 0 258 149
(147, 1500, 3750, 2000, 4250)
9 237 140 381
(148, 1500, 4000, 2000, 4500)
9 0 140 131
(158, 1750, 1000, 2250, 1500)
182 138 363 303
(188, 2000, 3000, 2500, 3500)
177 295 364 459
(189, 2000, 3250, 2500, 3750)
177 45 364 209
(203, 2250, 1250, 2750, 1750)
215 367 354 500
(204, 2250, 1500, 2750, 2000)
215 117 354 282
(223, 2500, 750, 3000, 1250)
282 294 436 447
(224, 2500, 1000, 3000, 1500)
282 44 436 197
(228, 2500, 200

(62, 500, 4500, 1000, 5000)
81 0 320 174
(72, 750, 1500, 1250, 2000)
61 375 183 500
(73, 750, 1750, 1250, 2250)
61 125 183 260
(74, 750, 2000, 1250, 2500)
362 298 491 417
(75, 750, 2250, 1250, 2750)
362 48 491 167
(80, 750, 3500, 1250, 4000)
269 288 460 470
(81, 750, 3750, 1250, 4250)
269 38 460 220
(83, 750, 4250, 1250, 4750)
324 235 489 390
(84, 750, 4500, 1250, 5000)
324 0 489 140
(93, 1000, 1250, 1500, 1750)
354 324 484 445
(94, 1000, 1500, 1500, 2000)
354 74 484 195
(96, 1000, 2000, 1500, 2500)
112 298 241 417
(97, 1000, 2250, 1500, 2750)
112 48 241 167
(101, 1000, 3250, 1500, 3750)
230 41 500 310
(102, 1000, 3500, 1500, 4000)
19 288 210 470
(103, 1000, 3750, 1500, 4250)
19 38 210 220
(103, 1000, 3750, 1500, 4250)
286 357 463 500
(104, 1000, 4000, 1500, 4500)
286 107 463 260
(105, 1000, 4250, 1500, 4750)
74 235 239 390
(106, 1000, 4500, 1500, 5000)
74 0 239 140
(111, 1250, 250, 1750, 750)
358 257 493 381
(112, 1250, 500, 1750, 1000)
358 7 493 131
(114, 1250, 1000, 1750, 1500)
317 

(23, 500, 1725, 1000, 2225)
238 185 332 279
(23, 500, 1725, 1000, 2225)
185 115 280 208
(23, 500, 1725, 1000, 2225)
140 7 236 100
(24, 750, 0, 1250, 500)
416 342 500 438
(24, 750, 0, 1250, 500)
361 276 447 369
(24, 750, 0, 1250, 500)
285 365 361 451
(24, 750, 0, 1250, 500)
320 436 399 500
(25, 750, 250, 1250, 750)
410 311 491 394
(25, 750, 250, 1250, 750)
416 92 500 188
(25, 750, 250, 1250, 750)
361 26 447 119
(25, 750, 250, 1250, 750)
285 115 361 201
(25, 750, 250, 1250, 750)
320 186 399 267
(25, 750, 250, 1250, 750)
362 248 439 334
(26, 750, 500, 1250, 1000)
352 316 450 423
(26, 750, 500, 1250, 1000)
291 243 386 339
(26, 750, 500, 1250, 1000)
410 61 491 144
(26, 750, 500, 1250, 1000)
362 0 439 84
(27, 750, 750, 1250, 1250)
352 66 450 173
(27, 750, 750, 1250, 1250)
291 0 386 89
(27, 750, 750, 1250, 1250)
198 195 298 282
(27, 750, 750, 1250, 1250)
102 247 213 352
(28, 750, 1000, 1250, 1500)
160 302 261 400
(28, 750, 1000, 1250, 1500)
379 380 472 469
(28, 750, 1000, 1250, 1500)
12 414 1

38 248 117 322
(116, 1750, 1000, 2250, 1500)
30 167 117 246
(116, 1750, 1000, 2250, 1500)
39 92 114 154
(116, 1750, 1000, 2250, 1500)
221 152 292 235
(116, 1750, 1000, 2250, 1500)
231 312 286 362
(117, 1750, 1250, 2250, 1750)
23 185 91 250
(117, 1750, 1250, 2250, 1750)
44 73 121 158
(117, 1750, 1250, 2250, 1750)
411 30 500 110
(117, 1750, 1250, 2250, 1750)
38 0 117 72
(117, 1750, 1250, 2250, 1750)
231 62 286 112
(120, 1750, 2000, 2250, 2500)
376 435 442 500
(121, 1750, 2250, 2250, 2750)
14 366 81 427
(121, 1750, 2250, 2250, 2750)
206 294 262 359
(121, 1750, 2250, 2250, 2750)
234 340 285 392
(121, 1750, 2250, 2250, 2750)
376 185 442 252
(122, 1750, 2500, 2250, 3000)
14 116 81 177
(122, 1750, 2500, 2250, 3000)
206 44 262 109
(122, 1750, 2500, 2250, 3000)
234 90 285 142
(124, 1750, 3000, 2250, 3500)
382 392 497 500
(125, 1750, 3250, 2250, 3750)
241 231 348 347
(125, 1750, 3250, 2250, 3750)
382 142 497 255
(126, 1750, 3500, 2250, 4000)
241 0 348 97
(131, 2000, 750, 2500, 1250)
349 406 433 

(186, 2750, 2500, 3250, 3000)
3 347 118 462
(187, 2750, 2750, 3250, 3250)
3 97 118 212
(194, 3000, 500, 3500, 1000)
412 338 463 381
(195, 3000, 750, 3500, 1250)
412 88 463 131
(196, 3000, 1000, 3500, 1500)
61 269 148 349
(196, 3000, 1000, 3500, 1500)
251 463 293 500
(196, 3000, 1000, 3500, 1500)
76 386 118 439
(196, 3000, 1000, 3500, 1500)
174 391 219 443
(196, 3000, 1000, 3500, 1500)
21 381 71 436
(197, 3000, 1250, 3500, 1750)
127 281 218 371
(197, 3000, 1250, 3500, 1750)
61 19 148 99
(197, 3000, 1250, 3500, 1750)
46 286 110 363
(197, 3000, 1250, 3500, 1750)
251 213 293 258
(197, 3000, 1250, 3500, 1750)
76 136 118 189
(197, 3000, 1250, 3500, 1750)
174 141 219 193
(197, 3000, 1250, 3500, 1750)
21 131 71 186
(197, 3000, 1250, 3500, 1750)
337 439 384 485
(198, 3000, 1500, 3500, 2000)
39 308 93 364
(198, 3000, 1500, 3500, 2000)
127 31 218 121
(198, 3000, 1500, 3500, 2000)
46 36 110 113
(198, 3000, 1500, 3500, 2000)
34 422 79 478
(198, 3000, 1500, 3500, 2000)
337 189 384 235
(198, 3000, 15

...

(98, 1500, 500, 2000, 1000)
133 30 248 139
(99, 1500, 750, 2000, 1250)
156 0 252 92
(100, 1500, 1000, 2000, 1500)
187 391 262 450
(100, 1500, 1000, 2000, 1500)
105 383 184 444
(100, 1500, 1000, 2000, 1500)
24 382 97 441
(101, 1500, 1250, 2000, 1750)
422 259 500 357
(101, 1500, 1250, 2000, 1750)
187 141 262 200
(101, 1500, 1250, 2000, 1750)
105 133 184 194
(101, 1500, 1250, 2000, 1750)
24 132 97 191
(102, 1500, 1500, 2000, 2000)
422 9 500 107
(102, 1500, 1500, 2000, 2000)
311 277 500 452
(103, 1500, 1750, 2000, 2250)
311 27 500 202
(106, 1500, 2500, 2000, 3000)
351 354 472 461
(107, 1500, 2750, 2000, 3250)
351 104 472 211
(117, 1750, 1250, 2250, 1750)
172 259 272 357
(118, 1750, 1500, 2250, 2000)
172 9 272 107
(118, 1750, 1500, 2250, 2000)
61 277 258 452
(119, 1750, 1750, 2250, 2250)
61 27 258 202
(121, 1750, 2250, 2250, 2750)
285 356 383 429
(122, 1750, 2500, 2250, 3000)
285 106 383 179
(122, 1750, 2500, 2250, 3000)
101 354 222 461
(123, 1750, 2750, 2250, 3250)
101 104 222 211
(137, 20

(276, 5250, 750, 5750, 1250)
0 417 60 483
(277, 5250, 1000, 5750, 1500)
0 167 60 233
(277, 5250, 1000, 5750, 1500)
13 228 48 265
包含样本49，标记样本312
P2691
(62, 500, 3500, 1000, 4000)
385 331 445 395
(63, 500, 3750, 1000, 4250)
385 81 445 145
(86, 750, 3500, 1250, 4000)
135 331 195 395
(87, 750, 3750, 1250, 4250)
135 81 195 145
(104, 1000, 2000, 1500, 2500)
248 402 277 433
(105, 1000, 2250, 1500, 2750)
248 152 277 183
(127, 1250, 1750, 1750, 2250)
463 375 494 408
(127, 1250, 1750, 1750, 2250)
357 412 383 438
(127, 1250, 1750, 1750, 2250)
427 282 457 311
(127, 1250, 1750, 1750, 2250)
336 395 362 422
(127, 1250, 1750, 1750, 2250)
313 376 341 404
(128, 1250, 2000, 1750, 2500)
463 125 494 158
(128, 1250, 2000, 1750, 2500)
357 162 383 188
(128, 1250, 2000, 1750, 2500)
427 32 457 61
(128, 1250, 2000, 1750, 2500)
368 471 395 498
(128, 1250, 2000, 1750, 2500)
349 451 376 479
(128, 1250, 2000, 1750, 2500)
455 471 479 499
(128, 1250, 2000, 1750, 2500)
381 419 408 445
(128, 1250, 2000, 1750, 2500)
362 

(49, 1250, 1000, 1750, 1500)
95 279 241 427
(49, 1250, 1000, 1750, 1500)
17 302 171 459
(50, 1250, 1250, 1750, 1750)
95 29 241 177
(50, 1250, 1250, 1750, 1750)
17 52 171 209
(66, 1750, 750, 2250, 1250)
120 150 295 336
(66, 1750, 750, 2250, 1250)
223 376 323 469
(67, 1750, 1000, 2250, 1500)
223 126 323 219
(81, 2250, 0, 2750, 500)
305 296 477 481
(82, 2250, 250, 2750, 750)
305 46 477 231
(90, 2486, 0, 2986, 500)
69 296 241 481
(91, 2486, 250, 2986, 750)
69 46 241 231
包含样本11，标记样本99
P2748
(0, 0, 0, 500, 500)
96 213 256 369
(0, 0, 0, 500, 500)
188 298 347 457
(0, 0, 0, 500, 500)
66 353 260 500
(0, 0, 0, 500, 500)
88 136 194 236
(0, 0, 0, 500, 500)
19 191 125 295
(0, 0, 0, 500, 500)
186 141 285 241
(1, 0, 250, 500, 750)
96 0 256 119
(1, 0, 250, 500, 750)
188 48 347 207
(1, 0, 250, 500, 750)
66 103 260 294
(19, 250, 1750, 750, 2250)
267 203 361 295
(31, 500, 1750, 1000, 2250)
17 203 111 295
(34, 500, 2500, 1000, 3000)
314 167 476 326
(35, 500, 2572, 1000, 3072)
314 95 476 254
(40, 750, 1000,

(113, 2000, 2250, 2500, 2750)
0 40 171 227
(153, 2750, 2500, 3250, 3000)
144 282 399 500
(154, 2750, 2750, 3250, 3250)
144 32 399 282
(163, 3000, 1750, 3500, 2250)
177 151 369 376
(219, 4000, 2750, 4500, 3250)
245 264 402 421
(220, 4000, 2923, 4500, 3423)
245 91 402 248
(232, 4127, 2750, 4627, 3250)
118 264 275 421
(233, 4127, 2923, 4627, 3423)
118 91 275 248
包含样本12，标记样本234
P2791
(4, 0, 1000, 500, 1500)
232 214 500 464
(25, 250, 250, 750, 750)
43 211 382 490
(26, 250, 500, 750, 1000)
107 181 377 448
(26, 250, 500, 750, 1000)
43 0 382 240
(27, 250, 750, 750, 1250)
10 200 330 461
(28, 250, 1000, 750, 1500)
10 0 330 211
(28, 250, 1000, 750, 1500)
0 214 306 464
(29, 250, 1250, 750, 1750)
0 0 306 214
(73, 750, 250, 1250, 750)
70 77 440 452
(126, 1250, 1500, 1750, 2000)
244 161 500 468
(131, 1250, 2750, 1750, 3250)
220 129 489 422
(150, 1500, 1500, 2000, 2000)
0 161 306 468
(151, 1500, 1750, 2000, 2250)
119 0 453 356
(155, 1500, 2750, 2000, 3250)
0 129 239 422
(175, 1750, 1750, 2250, 2250)
1

包含样本32，标记样本460
P2804
(8, 250, 0, 750, 500)
107 274 367 500
(9, 250, 250, 750, 750)
107 24 367 254
(18, 500, 500, 1000, 1000)
93 299 304 496
(19, 500, 750, 1000, 1250)
93 49 304 246
(20, 500, 1000, 1000, 1500)
301 220 500 498
(28, 750, 1000, 1250, 1500)
51 220 303 498
(29, 750, 1250, 1250, 1750)
51 0 303 248
(61, 1750, 1250, 2250, 1750)
248 243 376 379
(61, 1750, 1250, 2250, 1750)
359 198 500 371
(62, 1750, 1500, 2250, 2000)
248 0 376 129
(67, 2000, 750, 2500, 1250)
191 349 325 472
(67, 2000, 750, 2500, 1250)
296 301 496 500
(68, 2000, 1000, 2500, 1500)
117 155 351 386
(68, 2000, 1000, 2500, 1500)
191 99 325 222
(68, 2000, 1000, 2500, 1500)
296 51 496 253
(69, 2000, 1250, 2500, 1750)
0 243 126 379
(69, 2000, 1250, 2500, 1750)
109 198 261 371
(69, 2000, 1250, 2500, 1750)
236 252 383 420
(69, 2000, 1250, 2500, 1750)
315 304 437 415
(70, 2000, 1500, 2500, 2000)
0 0 126 129
(70, 2000, 1500, 2500, 2000)
236 2 383 170
(70, 2000, 1500, 2500, 2000)
315 54 437 165
(75, 2250, 750, 2750, 1250)
46 

In [22]:
[ x for x in range(0, 601, 250)]

[0, 250, 500]